# 2.6 Stokes equation

Find $u \in [H^1_D]^2$ and $p \in L_2$ such that

$$
\DeclareMathOperator{\Div}{div}
\begin{array}{ccccll}
\int \nabla u : \nabla v & + & \int \Div v \, p & = & \int f v & \forall \, v \\
\int \Div u \, q &  &  & = & 0 & \forall \, q
\end{array}
$$

Define channel geometry and mesh it:

In [1]:
from ngsolve import *
import netgen.gui
%gui tk

from netgen.geom2d import SplineGeometry
geo = SplineGeometry()
geo.AddRectangle( (0, 0), (2, 0.41), bcs = ("wall", "outlet", "wall", "inlet"))
geo.AddCircle ( (0.2, 0.2), r=0.05, leftdomain=0, rightdomain=1, bc="cyl")
mesh = Mesh( geo.GenerateMesh(maxh=0.05))
mesh.Curve(3)
Draw (mesh)

Use Taylor Hood finite element pairing: Continuous $P^2$ elements for velocity, and continuous $P^1$ for pressure:

In [2]:
V = H1(mesh, order=2, dirichlet="wall|inlet|cyl")
Q = H1(mesh, order=1)
X = FESpace([V,V,Q])

Setup bilinear-form for Stokes. We give names for all scalar field components. The divergence is constructed from partial derivatives of the velocity components.

In [3]:
ux,uy,p = X.TrialFunction()
vx,vy,q = X.TestFunction()

div_u = grad(ux)[0]+grad(uy)[1]
div_v = grad(vx)[0]+grad(vy)[1]

a = BilinearForm(X)
a += SymbolicBFI(grad(ux)*grad(vx)+grad(uy)*grad(vy) + div_u*q + div_v*p)
a.Assemble()

Set inhomogeneous Dirichlet boundary condition only on inlet boundary:

In [4]:
gfu = GridFunction(X)
uin = 1.5*4*y*(0.41-y)/(0.41*0.41)
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))
velocity = CoefficientFunction(gfu.components[0:2])
Draw(velocity, mesh, "vel")
Draw(Norm(velocity), mesh, "|vel|")
SetVisualization(max=2)

Solve equation:

In [5]:
res = gfu.vec.CreateVector()
res.data = -a.mat * gfu.vec
inv = a.mat.Inverse(freedofs=X.FreeDofs(), inverse="umfpack")
gfu.vec.data += inv * res
Redraw()

## Testing different velocity-pressure pairs
Now we define a Stokes setup function to test different spaces:

In [6]:
def SolveStokes(X):
    ux,uy,p = X.TrialFunction()
    vx,vy,q = X.TestFunction()
    div_u = grad(ux)[0]+grad(uy)[1]
    div_v = grad(vx)[0]+grad(vy)[1]
    a = BilinearForm(X)
    a += SymbolicBFI(grad(ux)*grad(vx)+grad(uy)*grad(vy) + div_u*q + div_v*p)
    a.Assemble()
    gfu = GridFunction(X)
    uin = 1.5*4*y*(0.41-y)/(0.41*0.41)
    gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))
    res = gfu.vec.CreateVector()
    res.data = -a.mat * gfu.vec
    inv = a.mat.Inverse(freedofs=X.FreeDofs(), inverse="umfpack")
    gfu.vec.data += inv * res
    

    velocity = CoefficientFunction(gfu.components[0:2])
    Draw(velocity, mesh, "vel")
    Draw(Norm(velocity), mesh, "|vel|")
    SetVisualization(max=2)    
    
    return gfu

Higher order Taylor-Hood elements:

In [7]:
V = H1(mesh, order=4, dirichlet="wall|inlet|cyl")
Q = H1(mesh, order=3)
X = FESpace([V,V,Q])

gfu = SolveStokes(X)

With discontinuous pressure elements P2-P1 is unstable:

In [8]:
V = H1(mesh, order=2, dirichlet="wall|inlet|cyl")
Q = L2(mesh, order=1)
print ("V.ndof =", V.ndof, ", Q.ndof =", Q.ndof)
X = FESpace([V,V,Q])

gfu = SolveStokes(X)

V.ndof = 1702 , Q.ndof = 2382


RuntimeError: UmfpackInverse: Numeric factorization failed.

$P^{2,+} \times P^{1,dc}$ elements:

In [9]:
V = H1(mesh, order=2, dirichlet="wall|inlet|cyl")
V.SetOrder(TRIG,3)
V.Update()
Q = L2(mesh, order=1)
X = FESpace([V,V,Q])
print ("V.ndof =", V.ndof, ", Q.ndof =", Q.ndof)

gfu = SolveStokes(X)

V.ndof = 2496 , Q.ndof = 2382


the mini element:

In [10]:
V = H1(mesh, order=1, dirichlet="wall|inlet|cyl")
V.SetOrder(TRIG,3)
V.Update()
Q = H1(mesh, order=1)
X = FESpace([V,V,Q])

gfu = SolveStokes(X)

## VectorH1 

A vector-valued $H^1$-space: Less to type and more possibilities to explore structure and optimize. 

In [11]:
V = VectorH1(mesh, order=2, dirichlet="wall|inlet|cyl")
V.SetOrder(TRIG,3)
V.Update()
Q = L2(mesh, order=1)
X = FESpace([V,Q])

u,p = X.TrialFunction()
v,q = X.TestFunction()

a = BilinearForm(X)
a += SymbolicBFI(InnerProduct(grad(u),grad(v))+div(u)*q+div(v)*p)
a.Assemble()

gfu = GridFunction(X)
uin = CoefficientFunction( (1.5*4*y*(0.41-y)/(0.41*0.41), 0) )
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))

res = gfu.vec.CreateVector()
res.data = -a.mat * gfu.vec
inv = a.mat.Inverse(freedofs=X.FreeDofs(), inverse="umfpack")
gfu.vec.data += inv * res
Draw(gfu.components[0], mesh, "vel")
Draw(Norm(gfu.components[0]), mesh, "|vel|")
SetVisualization(max=2)

## Stokes as a block-system
We can now define separate bilinear-form and matrices for A and B, and combine them to a block-system:

In [12]:
V = VectorH1(mesh, order=3, dirichlet="wall|inlet|cyl")
Q = H1(mesh, order=2)

u,v = V.TnT()
p,q = Q.TnT()

a = BilinearForm(V)
a += SymbolicBFI(InnerProduct(grad(u),grad(v)))

b = BilinearForm(trialspace=V, testspace=Q)
b += SymbolicBFI(div(u)*q)

a.Assemble()
b.Assemble()

Needed as preconditioner for the pressure:

In [13]:
mp = BilinearForm(Q)
mp += SymbolicBFI(p*q)
mp.Assemble()

Two right hand sides for the two spaces:

In [14]:
f = LinearForm(V)
f += SymbolicLFI( CoefficientFunction( (0,x-0.5)) * v)
f.Assemble()

g = LinearForm(Q)
g.Assemble()

Two `GridFunction`s for velocity and pressure:

In [15]:
gfu = GridFunction(V, name="u")
gfp = GridFunction(Q, name="p")
uin = CoefficientFunction( (1.5*4*y*(0.41-y)/(0.41*0.41), 0) )
gfu.Set(uin, definedon=mesh.Boundaries("inlet"))

Combine everything to a block-system.
`BlockMatrix` and `BlockVector` store references to the original matrices and vectors, no new large matrices are allocated. The same for the transpose matrix `b.mat.T`. It stores a wrapper for the original matrix, and replaces the call of the `Mult` function by `MultTrans`.

In [16]:
K = BlockMatrix( [ [a.mat, b.mat.T], [b.mat, None] ] )
C = BlockMatrix( [ [a.mat.Inverse(V.FreeDofs()), None], [None, mp.mat.Inverse()] ] )

rhs = BlockVector ( [f.vec, g.vec] )
sol = BlockVector( [gfu.vec, gfp.vec] )

solvers.MinRes (mat=K, pre=C, rhs=rhs, sol=sol, initialize=False)

it =  0  err =  4.577250004746686
it =  1  err =  2.3137027107825414
it =  2  err =  1.9474651703872594
it =  3  err =  1.5362811821422817
it =  4  err =  1.490898743198753
it =  5  err =  1.277851941805208
it =  6  err =  1.236894318332737
it =  7  err =  1.0832164597336917
it =  8  err =  0.9954867877973922
it =  9  err =  0.8694764391752701
it =  10  err =  0.8164294710037596
it =  11  err =  0.7270898498126048
it =  12  err =  0.7062300357748325
it =  13  err =  0.6435150710145074
it =  14  err =  0.6260336874381139
it =  15  err =  0.591910771493464
it =  16  err =  0.577362791218001
it =  17  err =  0.5490895491963667
it =  18  err =  0.5299584240360132
it =  19  err =  0.504587770754635
it =  20  err =  0.4945747930809748
it =  21  err =  0.478452427389186
it =  22  err =  0.46049192930432753
it =  23  err =  0.4367980978458769
it =  24  err =  0.41255367261726367
it =  25  err =  0.38040899835370073
it =  26  err =  0.3449005478686267
it =  27  err =  0.2985421196154886
it =  2

basevector

In [17]:
Draw (gfu)